<a href="https://colab.research.google.com/github/kaballas/AISECKG-QA-Dataset/blob/main/the_model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import math


"""
Input Embeddings are the first step to build a transformer.
We set up the dimension of the embeddings (d_model),
and the vocab size (vocan_size). Then we use an embedding
layer from torch.nn and set num_embeddings and embedding_dim
"""
class InputEmbeddings(nn.Module):

    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model
        self.voca_size = vocab_size
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=d_model
        )

    def forward(self, x):
        # Multiply the embeddings by the squared root of the d_model (Vaswani et al. 2017).
        return self.embedding(x) * math.sqrt(self.d_model)


"""
Because the attention mechanisms are position invariant, we need
to encode the information of word order in some way. The
authors came up with positional encoding, which is summed to
the original embeddings.
"""
class PositionalEmbeddings(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float):
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout= nn.Dropout(dropout)

        # Create a tensor of zeros to fill in
        pe = torch.zeros(seq_len, d_model)
        # Create a positions vector of shape seq_len. We use .arange()
        position = torch.arange(start=0, end=seq_len, dtype=torch.float).unsqueeze(1)
        # We create the division term of the formula
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000) / d_model))
        # Apply sine and cosine. The sine is applied to even numbers; cosine to odd numbers
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # Now we need to make it applicable to batches. To do so, we need
        # to add an extra dimension in the first position. We do so by using
        # .unsqueeze at position 0
        pe = pe.unsqueeze(0)

        # We need to register the tensor in the buffer of the module (kind of remember it).
        self.register_buffer('pe', pe)

    def forward(self, x):
        # We add the positional encoding to the input tensor. We slice it to match
        # the dimensions of the word embedding. Remember the dimensions are: [batch, pos_embedings, dimension]
        # We take dim 1 (pos_embeddings) and align them with the dim 1 of x (the
        # actual word embeddings). We also make sure to make the positional
        # embeddings static (.requires_grad).
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad(False)
        return self.dropout(x)


"""
We now design the Layer Normalization. We do this by passing an epsilon value (eps),
which is added to avoid 0 division in the normalization operation.
"""
class LayerNormalization(nn.Module):

    def __init__(self, eps: float = 10 ** -6):
        super().__init__()
        self.eps = eps
        # We set alpha and bias as trainable params
        self.alpha = nn.Parameter(torch.ones(1))
        self.bias  = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        # Get mean and std
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)
        # Finally use the formula to normalize
        return self.alpha * (x - mean) / (std + self.eps) + self.bias


"""
This is the Position Wise Feed-Forward Network. That's just a fancy name
for a simple neural network, with two layers and a Relu activation in between.
"""
class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff: int, dropout: float):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff) # This corresponds to W1 and B1 of Vaswani et al. (2017)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_ff, d_model) # This corresponds to W2 and B2

    def forward(self, x):
        # Pass the main function and return [d_model] dim
        return self.linear2(self.dropout(nn.ReLU(self.linear1(x))))


"""
This is probably the most important block: the famous Multi-Head Attention.
"""
class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, n_heads: int, dropout: float):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads

        # We must be able to send equal dims to the different heads
        assert d_model % n_heads == 0, "Division between d_model and n_heads must be possible."

        # d_k is the dim that each tensor will have to be parallelized in different heads
        self.d_k = d_model // n_heads

        # Set up the weight matrices
        self.w_q = nn.Linear(d_model, d_model, bias=False) # Wq
        self.w_k = nn.Linear(d_model, d_model, bias=False) # Wk
        self.w_v = nn.Linear(d_model, d_model, bias=False) # Wv

        # Set Wo, which is [(n_heads * self.d_v), d_model] = [d_model, d_model]
        self.w_o = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)

        """
        COMMON QUESION: If d_k and d_v are the same dimensions,
        why do they have different names? d_v is the result of the last
        multiplication of the attention formula (which is by V; see
        the original paper). However, in practice they are the same.
        """


    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        # We extact the d_k, which is the last dimension of Q, K, and V
        d_k = query.shape[-1]
        # We apply the self-attention scores formula. We transpose the last two dims to make the calculation possible
        # Transform: [Batch, n_heads, seq_len, d_k] -> [Batch, n_heads, seq_len, seq_len]
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        # This is for the masking
        if mask is not None:
            attention_scores.masked_fill_(mask == 0, -1e9)
        # We apply softmax
        attention_scores = attention_scores.softmax(dim = -1)
        # We add dropout
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        # We return a tupple with the attention and the self-attention for visualization
        return (attention_scores @ value), attention_scores


    def forward(self, q, k, v, mask):
        # Project the embeddings into the weight matrices [Batch, seq_len, d_model] -> [Batch, seq_len, d_model]
        query = self.w_q(q)
        key = self.w_k(k)
        value = self.w_v(v)

        """
        IMPORTANT:
        We need to divide Q, K and V to feed "pieces" to different heads (power of parallel processing!)
        Transform: [Batch, seq_len, d_model] -> [Batch, seq_len, n_heads, d_k] -> [Batch, n_heads, seq_len x d_k]

            - We don't want to split the batches: query.shape[0]
            - We don't want to split the sequence: query.shape[1]
            - We want to split the d_model (embeddings): self.n_heads, self.d_k
            - We want the transposition because we want each head to see the seq_len and d_k

        The transposition allows the model to process each head independently across the sequence length. Each head can
        focus on different parts of the input sequence, enabling the model to capture various aspects of
        the input data in parallel.
        """
        query = query.view(query.shape[0], query.shape[1], self.n_heads, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.n_heads, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.n_heads, self.d_k).transpose(1, 2)

        # here we use the function we previously introduced as a static method
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

        # Here we concatenate the information from the different heads. By multiplying n_heads and d_k,
        # we effectively concatenate the d_k elements from each head for every sequence position into a single
        # dimension, resulting in a tensor where the information from different heads is concatenated
        # for each position in the sequence.
        # Transform: [batch, n_heads, seq_len, d_k] -> [batch, seq_len, n_heads, d_k] -> [batch, seq_len, d_model]
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.n_heads * self.d_k)

        # Project x it in weights Wo and return it as [batch, seq_len, d_model]
        return self.w_o(x)


"""
Here we will build the residual connection component of the transformer.
This will allow a better training and make some 'raw' input flow from layer to layer.
"""
class ResidualConnection(nn.Module):

    def __init__(self, dropout: float):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.normalization = LayerNormalization()

    def forward(self, x, sublayer):
        # Normalize x, then pass it through a sublayer (any type), use the dropout term, and finally add x
        return x + self.dropout(sublayer(self.normalization(x)))


"""
Here's the encoder block that we will use to create the
Encoder object (stacking of Encoder layers)
"""
class EncoderBlock(nn.Module):

    def __init__(self, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float):
        super().__init__()
        # This is the multi-head attention
        self.self_attention_block = self_attention_block
        # This is the Point-wise feed forward network
        self.feed_forward_block = feed_forward_block
        # we pack two residual connections in a nn.ModuleList
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        # In the first residual connection (idx: 0), we are using MultiHeadAttention, which takes Q, K, V and mask, and
        # the residual input. We add both and we pass that result to the second residual connection (idx: 1). This
        # makes the same operation but with the FeedForwardBlock.
        x = self.residual_connections[0](lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x


"""
This is how we stack the Encoder block in several layers. This will be
the main Encoder object. Because it has to be able to take several Encoder
blocks, we use nn.ModuleList as a parameter
"""
class Encoder(nn.Module):

    def __init__(self, n_layers: nn.ModuleList):
        super().__init__()
        # We can create as many layers as we want
        self.n_layers = n_layers
        self.normalization = LayerNormalization()

    def forward(self, x, mask):
        # we iterate over n layers
        for layer in self.n_layers:
            x = layer(x, mask)
        # finally we normalize
        return self.normalization(x)


"""
This will be the Decoder block that will allow us to make several layers of it.
We introduce cross attention, which is similar to multi-head attention but taken
parameters from the encoder.
"""
class DecoderBlock(nn.Module):

    def __init__(self, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock,
                 feed_forward_block: FeedForwardBlock, dropout: float):
        super().__init__()
        # Initialize all the pieces
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        # Get three residual connections
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)])

    def forward(self, x, encoder_out, src_mask, tgt_mask):
        x = self.residual_connections[0](lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](lambda x: self.self_attention_block(x, encoder_out, encoder_out, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)


"""
This will be our main Decoder object that we will use to ensemble
all layers.
"""
class Decoder(nn.Module):

    def __init__(self, n_layers: nn.ModuleList):
        super().__init__()
        # we set the n_layers parameter
        self.n_layers = n_layers
        self.normalization = LayerNormalization()

    def forward(self, x, enc_out, src_mask, tgt_mask):
        # we iterate over n layers
        for layer in self.n_layers:
            x = layer(x, enc_out, src_mask, tgt_mask)
        # finally we normalize
        return self.normalization(x)


"""
Final layer to convert logits to a probability distribution over all the vocabulary
of the target language (we are building the og transformer)
"""
class LastLinear(nn.Module):

    def __init__(self, d_model, vocab_size):
        super().__init__()
        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        return torch.log_softmax(self.fc(x), dim=-1)


"""
This is the actual beast: the Transformer. We use all the blocks we have been
carefully assembling to build the following class
"""
class Transformer(nn.Module):

    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings,
                 tgt_embed: InputEmbeddings, src_pos: PositionalEmbeddings, tgt_pos: PositionalEmbeddings,
                 last_linear: LastLinear):
        super().__init__()
        # Set all the components we will need
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.last_linear = last_linear

    # We define an encode() function for the encoder.
    def encode(self, src, src_mask):
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)

    # We define a decode() function for the decoder.
    def decode(self, enc_out, src_mask, tgt, tgt_mask):
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, enc_out, src_mask, tgt_mask)

    # We pass the outputs through the final linear layer
    def linear(self, x):
        return self.last_linear(x)


"""
We will finally create a function that allows us to ensemble everything
together and create the layers of encoders and decoders. This is what the
original paper calls Nx in the figure to the sides of the main figure.
"""
def build_transformer_model(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int,
                            tgt_seq_len: int, d_model: int= 512, n_layers: int = 6,
                            n_heads: int = 8, dropout: float = 0.1, hidden_size: int = 2048):

    # Make the embedding layers for input and target
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Make the positional embeddings for input and target
    # (in practice you can use the same for both src and tgt)
    src_pos = PositionalEmbeddings(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEmbeddings(d_model, tgt_seq_len, dropout)

    # create the encoder blocks
    encoder_blocks = []
    for _ in range(n_layers):
        encoder_self_attention = MultiHeadAttentionBlock(d_model, n_heads, dropout)
        feed_forward = FeedForwardBlock(d_model, hidden_size, dropout)
        encoder_block = EncoderBlock(encoder_self_attention, feed_forward, dropout)
        encoder_blocks.append(encoder_block)

    # create the decoder blocks
    decoder_blocks = []
    for _ in range(n_layers):
        decoder_self_attention = MultiHeadAttentionBlock(d_model, n_heads, dropout)
        decoder_cross_attention = MultiHeadAttentionBlock(d_model, n_heads, dropout)
        feed_forward = FeedForwardBlock(d_model, hidden_size, dropout)
        decoder_block = DecoderBlock(decoder_self_attention, decoder_cross_attention, feed_forward, dropout)
        decoder_blocks.append(decoder_block)

    """
    Now we pass those layers as the argumemtn to the main objects.
    Remember that our main classes for encoder and decoder take
    nn.ModuleList as arguments (aka, the layers stacked)
    """
    encoder = Encoder(nn.ModuleList(encoder_blocks))
    decoder = Decoder(nn.ModuleList(decoder_blocks))

    last_layer = LastLinear(d_model, tgt_vocab_size)

    # Now, build our model using the transformer class we built
    transformer = Transformer(
        encoder=encoder,
        decoder=decoder,
        src_embed=src_embed,
        tgt_embed=tgt_embed,
        src_pos=src_pos,
        tgt_pos=tgt_pos,
        last_linear=last_layer
    )

    # Now we initialize the parameters with Xavier initialization
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    # FINALLY, return the transformer
    return transformer

In [2]:
!pip install datasets

In [2]:


!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
     

In [1]:
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, random_split
import math

# Define model components
class InputEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model)

class PositionalEmbeddings(nn.Module):
    def __init__(self, d_model: int, max_len: int, dropout: float):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

class LayerNormalization(nn.Module):
    def __init__(self, eps: float = 1e-6):
        super().__init__()
        self.alpha = nn.Parameter(torch.ones(1))
        self.bias = nn.Parameter(torch.zeros(1))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)
        return self.alpha * (x - mean) / (std + self.eps) + self.bias

class FeedForwardBlock(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.linear2(self.dropout(nn.ReLU()(self.linear1(x))))

class MultiHeadAttentionBlock(nn.Module):
    def __init__(self, d_model: int, n_heads: int, dropout: float):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        assert d_model % n_heads == 0
        self.d_k = d_model // n_heads

        self.w_q = nn.Linear(d_model, d_model, bias=False)
        self.w_k = nn.Linear(d_model, d_model, bias=False)
        self.w_v = nn.Linear(d_model, d_model, bias=False)
        self.w_o = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout):
        d_k = query.size(-1)
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        p_attn = scores.softmax(dim=-1)
        if dropout is not None:
            p_attn = dropout(p_attn)
        return torch.matmul(p_attn, value), p_attn

    def forward(self, q, k, v, mask):
        batch_size = q.size(0)
        q = self.w_q(q).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        k = self.w_k(k).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        v = self.w_v(v).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)

        scores, attn = self.attention(q, k, v, mask, self.dropout)
        scores = scores.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        return self.w_o(scores)

class ResidualConnection(nn.Module):
    def __init__(self, size, dropout):
        super().__init__()
        self.norm = LayerNormalization(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderBlock(nn.Module):
    def __init__(self, size, self_attention, feed_forward, dropout):
        super().__init__()
        self.self_attention = self_attention
        self.feed_forward = feed_forward
        self.sublayer = nn.ModuleList([ResidualConnection(size, dropout) for _ in range(2)])
        self.size = size

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attention(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

class Encoder(nn.Module):
    def __init__(self, layer, n_layers):
        super().__init__()
        self.layers = nn.ModuleList([layer for _ in range(n_layers)])
        self.norm = LayerNormalization(layer.size)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class DecoderBlock(nn.Module):
    def __init__(self, size, self_attention, cross_attention, feed_forward, dropout):
        super().__init__()
        self.self_attention = self_attention
        self.cross_attention = cross_attention
        self.feed_forward = feed_forward
        self.sublayer = nn.ModuleList([ResidualConnection(size, dropout) for _ in range(3)])
        self.size = size

    def forward(self, x, memory, src_mask, tgt_mask):
        x = self.sublayer[0](x, lambda x: self.self_attention(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.cross_attention(x, memory, memory, src_mask))
        return self.sublayer[2](x, self.feed_forward)

class Decoder(nn.Module):
    def __init__(self, layer, n_layers):
        super().__init__()
        self.layers = nn.ModuleList([layer for _ in range(n_layers)])
        self.norm = LayerNormalization(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

class LastLinear(nn.Module):
    def __init__(self, d_model, vocab_size):
        super().__init__()
        self.linear = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        return torch.log_softmax(self.linear(x), dim=-1)

class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, generator):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.generator = generator

    def encode(self, src, src_mask):
        return self.encoder(self.src_pos(self.src_embed(src)), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_pos(self.tgt_embed(tgt)), memory, src_mask, tgt_mask)

    def forward(self, src, tgt, src_mask, tgt_mask):
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def generate(self, x):
        return self.generator(x)

def build_transformer_model(src_vocab_size, tgt_vocab_size, max_len, d_model=512, n_layers=6, n_heads=8, dropout=0.1, d_ff=2048):
    attn = MultiHeadAttentionBlock(d_model, n_heads, dropout)
    ff = FeedForwardBlock(d_model, d_ff, dropout)
    position = PositionalEmbeddings(d_model, max_len, dropout)
    encoder_layer = EncoderBlock(d_model, attn, ff, dropout)
    decoder_layer = DecoderBlock(d_model, attn, attn, ff, dropout)

    encoder = Encoder(encoder_layer, n_layers)
    decoder = Decoder(decoder_layer, n_layers)

    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    model = Transformer(
        encoder=encoder,
        decoder=decoder,
        src_embed=src_embed,
        tgt_embed=tgt_embed,
        src_pos=position,
        tgt_pos=position,
        generator=LastLinear(d_model, tgt_vocab_size)
    )

    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return model


## Load dataset and tokenizer
#dataset = load_dataset("Kaballas/100", split="train")
dataset = load_dataset("Kaballas/HRMIS_MASTER", split="train")

# Keep only the columns you want
dataset = dataset.remove_columns([col for col in dataset.column_names if col not in ['questions', 'answers']])

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenization and encoding function
def encode(examples):
    # Encoding only 'instruction' and 'output'
    input_enc = tokenizer(examples['questions'], padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    output_enc = tokenizer(examples['answers'], padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    return {
        "input_ids": input_enc['input_ids'].squeeze(0),
        "attention_mask": input_enc['attention_mask'].squeeze(0),
        "output_ids": output_enc['input_ids'].squeeze(0)
    }

# Apply the encoding function
encoded_dataset = dataset.map(encode, batched=True, remove_columns=dataset.column_names)

# Define the PyTorch Dataset
class Seq2SeqDataset(torch.utils.data.Dataset):
    def __init__(self, encoded_dataset):
        self.encoded_dataset = encoded_dataset

    def __len__(self):
        return len(self.encoded_dataset)

    def __getitem__(self, idx):
        return {key: torch.tensor(self.encoded_dataset[idx][key]) for key in self.encoded_dataset[idx]}

# Create the dataset object
full_dataset = Seq2SeqDataset(encoded_dataset)

# Split the dataset
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Parameters
src_vocab_size = len(tokenizer)
tgt_vocab_size = len(tokenizer)
max_len = 128
print(f"Tokenizer src_vocab_size size: {src_vocab_size}")
print(f"Tokenizer tgt_vocab_size size: {tgt_vocab_size}")

# Build the model
model = build_transformer_model(src_vocab_size, tgt_vocab_size, max_len)
model.load_state_dict(torch.load('best_model.pt'))
model.to(device)

# Optimizer and Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

def create_masks(src, trg):
    src_mask = (src != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)
    trg_pad_mask = (trg != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)
    seq_len = trg.size(1)
    nopeak_mask = torch.tril(torch.ones((seq_len, seq_len), device=device)).bool()
    trg_mask = trg_pad_mask & nopeak_mask
    return src_mask, trg_mask

def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    accumulation_steps = 4
    optimizer.zero_grad()
    for i, batch in enumerate(iterator):
        src = batch['input_ids'].to(device)
        trg = batch['output_ids'].to(device)
        src_mask, trg_mask = create_masks(src, trg)
        enc_output = model.encode(src, src_mask)
        dec_output = model.decode(enc_output, src_mask, trg[:, :-1], trg_mask[:, :, :-1, :-1])
        output = model.generate(dec_output)
        output_dim = output.shape[-1]
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:, 1:].contiguous().view(-1)
        loss = criterion(output, trg)
        loss = loss / accumulation_steps
        loss.backward()
        if (i + 1) % accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            optimizer.zero_grad()
        epoch_loss += loss.item() * accumulation_steps
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch['input_ids'].to(device)
            trg = batch['output_ids'].to(device)
            src_mask, trg_mask = create_masks(src, trg)
            enc_output = model.encode(src, src_mask)
            dec_output = model.decode(enc_output, src_mask, trg[:, :-1], trg_mask[:, :, :-1, :-1])
            output = model.generate(dec_output)
            output_dim = output.shape[-1]
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:, 1:].contiguous().view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

# Training loop
num_epochs = 1
patience = 3
best_loss = float('inf')
counter = 0
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1)

for epoch in range(num_epochs):
    optimizer.lr = 0.00001  # Reduce the learning rate
    train_loss = train(model, train_loader, optimizer, criterion, clip=1)
    print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.3f}')
    val_loss = evaluate(model, val_loader, criterion)
    print(f'Epoch: {epoch+1}, Validation Loss: {val_loss:.3f}')
    scheduler.step(val_loss)
    if val_loss < best_loss:
        best_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping")
            #break
    if (epoch + 1) % 5 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': train_loss,
        }, f'checkpoint_epoch_{epoch+1}.pt')
        torch.save(model.state_dict(), 'best_model.pt')

Using the latest cached version of the dataset since Kaballas/HRMIS_MASTER couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/Kaballas___hrmis_master/default/0.0.0/ea8d856368b3d5ab80c1fed49e21c6335f7b4465 (last modified on Sat Jul 27 06:51:50 2024).


Tokenizer src_vocab_size size: 30522
Tokenizer tgt_vocab_size size: 30522
Epoch: 1, Train Loss: 6.136
Epoch: 1, Validation Loss: 6.147


In [2]:
# Generate text
def generate_text(model, tokenizer, input_text, max_len, temperature=0.7):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    input_mask = (input_ids != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)
    enc_output = model.encode(input_ids, input_mask)
    tgt_ids = torch.tensor([[tokenizer.bos_token_id or tokenizer.pad_token_id]]).to(device)
    tgt_mask = torch.zeros((1, 1, 1, 1)).to(device)
    for i in range(max_len):
        dec_output = model.decode(enc_output, input_mask, tgt_ids, tgt_mask)
        output = model.generate(dec_output[:, -1, :])
        next_word = torch.multinomial(torch.softmax(output / temperature, dim=-1), num_samples=1)
        tgt_ids = torch.cat((tgt_ids, next_word), dim=1)
        tgt_mask = (tgt_ids != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)
    return tokenizer.decode(tgt_ids[0], skip_special_tokens=True)

# Test the function
input_text = "what data is used for playbacks 1 and 2 in the data environment?"
output_text = generate_text(model, tokenizer, input_text, max_len=128)
print(output_text)

the. be stage the and - byst hr the. employee the central, the the change with the'process the. reporting and the the the time _ to the the purpose.. that the, the. i to a of the cash reports, of.ed the. the the of the in people y in the, of the. and in.g, the. in to the employee. in and is,oit work, to the the is as j of used the purpose, the is -mis. - the purposemis the hr hiring -, leave that identify
